In [73]:
import sys,os

sys.path.append('/home/arash/VRdataCleaning/DeepSurv/')

import importlib
import deepsurv
from statsmodels.stats.outliers_influence import variance_inflation_factor    
import argparse
import uuid
import pickle
import json

import numpy as np
import lasagne
import optunity

import logging
from logging import handlers


importlib.reload(deepsurv)

from deepsurv import deep_surv, utils, viz

from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger



import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
data = pd.read_pickle('ALLDATA.pkl')
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_40 - 49', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'shopwalk_kid', 'Vrexp_Yes', 'Heart_Currently',
                   'Heart_Over the years', 'vision_Currently', 
                   'vision_Over the years', 'anxiety_Currently', 'anxiety_Over the years', 'Headaches_Currently',
                   'Headaches_Over the years', 'dizziness_Over the years']]         #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later


data=data.apply(pd.to_numeric, errors='coerce')

data['E']=1   #all pedestrians cross, so no right censored data

In [75]:
def dataframe_to_deepsurv_ds(df, event_col = 'E', time_col = 'Wait Time (s)'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }


data = dataframe_to_deepsurv_ds(data, event_col = 'E', time_col= 'Wait Time (s)')

In [4]:











#Model with specific hyper parameters  #deepsurv needs to be fixed before use
hyperparams = {
    'L2_reg': 3.0,
    'batch_norm': False,
    'dropout': 0.02,
    'hidden_layers_sizes': [90,90,90],
    'learning_rate': 1e-05,
    'lr_decay': 0.001,
    'momentum': 0.9,
    'n_in': 10,
    'standardize': True
}

# Create an instance of DeepSurv using the hyperparams defined above
model = deep_surv.DeepSurv(**hyperparams)



experiment_name = 'Wait Time analysis'
logdir = './logs/tensorboard/'
logger = TensorboardLogger(experiment_name, logdir=logdir)

# Now we train the model
update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. \
                                            # Check out http://lasagne.readthedocs.io/en/latest/modules/updates.html \
                                            # for other optimizers to use
n_epochs = 2000

# If you have validation data, you can add it as the second parameter to the function
metrics = model.train(data, n_epochs=n_epochs, logger=logger, update_fn=update_fn)

# Print the final metrics
print('Train C-Index:', metrics['c-index'][-1])
# print('Valid C-Index: ',metrics['valid_c-index'][-1])

# Plot the training / validation curves
viz.plot_log(metrics)






ValueError: shapes (2463,47) and (10,90) not aligned: 47 (dim 1) != 10 (dim 0)
Apply node that caused the error: dot(x, W)
Toposort index: 13
Inputs types: [TensorType(float32, matrix), TensorType(float64, matrix)]
Inputs shapes: [(2463, 47), (10, 90)]
Inputs strides: [(188, 4), (720, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Add}[(0, 0)](dot.0, InplaceDimShuffle{x,0}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-4-a1391dd34a76>", line 41, in <module>
    metrics = model.train(data, n_epochs=n_epochs, logger=logger, update_fn=update_fn)
  File "/home/arash/VRdataCleaning/DeepSurv/deepsurv/deep_surv.py", line 431, in train
    update_fn = update_fn, **kwargs
  File "/home/arash/VRdataCleaning/DeepSurv/deepsurv/deep_surv.py", line 263, in _get_train_valid_fn
    learning_rate=learning_rate, **kwargs
  File "/home/arash/VRdataCleaning/DeepSurv/deepsurv/deep_surv.py", line 210, in _get_l

In [76]:
def load_logger(logdir):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    # Print to Stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # Print to Log file
    fh = logging.FileHandler(os.path.join(logdir, 'log_' + str(uuid.uuid4())))
    fh.setFormatter(format)
    logger.addHandler(fh)

    return logger

def load_box_constraints(file):
    with open(file, 'rb') as fp:
        return json.loads(fp.read())

def save_call_log(file, call_log):
    with open(file, 'wb') as fp:
        pickle.dump(call_log, fp)

In [87]:
def get_objective_function(num_epochs, logdir, update_fn = lasagne.updates.sgd):
    '''
    Returns the function for Optunity to optimize. The function returned by get_objective_function
    takes the parameters: x_train, y_train, x_test, and y_test, and any additional kwargs to 
    use as hyper-parameters.

    The objective function runs a DeepSurv model on the training data and evaluates it against the
    test set for validation. The result of the function call is the validation concordance index 
    (which Optunity tries to optimize)
    '''
    def format_to_deepsurv(x, y):
        return {
            'x': x,
            'e': y[:,0].astype(np.int32),
            't': y[:,1].astype(np.float32)
        }

    def get_hyperparams(params):
        hyperparams = {
            'batch_norm': False,
            'activation': 'rectify',
            'standardize': True
        }
        # @TODO add default parameters and only take necessary args from params
        # protect from params including some other key

        if 'num_layers' in params and 'num_nodes' in params:
            params['hidden_layers_sizes'] = [int(params['num_nodes'])] * int(params['num_layers'])
            del params['num_layers']
            del params['num_nodes']

        if 'learning_rate' in params:
            params['learning_rate'] = 10 ** params['learning_rate']
            
        if 'n_in' in params:
            params['n_in'] = int(params['n_in'])
            

        hyperparams.update(params)
        return hyperparams

    def train_deepsurv(x_train,y_train,x_test,y_test,
        **kwargs):
        hyperparams = get_hyperparams(kwargs)
        n_in=int(hyperparams['n_in'])

        #select number of features
        ReliefAvg = pd.read_pickle('/home/arash/VRdataCleaning/ReliefAvg.pkl')
        x_train=x_train[:,ReliefAvg.sort_values(['Importance'],ascending=False).index[0:n_in]]
        x_test=x_test[:,ReliefAvg.sort_values(['Importance'],ascending=False).index[0:n_in]]
        



        
        
        # Standardize the datasets
        train_mean = x_train.mean(axis = 0)
        train_std = x_train.std(axis = 0)

        x_train = (x_train - train_mean) / train_std
        x_test = (x_test - train_mean) / train_std

        train_data = format_to_deepsurv(x_train, y_train)
        valid_data = format_to_deepsurv(x_test, y_test)

        

        # Set up Tensorboard loggers
        # TODO improve the model_id for Tensorboard to better partition runs
        model_id = str(hash(str(hyperparams)))
        run_id = model_id + '_' + str(uuid.uuid4())
        logger = TensorboardLogger('hyperparam_search', 
            os.path.join(logdir,"tensor_logs", model_id, run_id))

        network = deep_surv.DeepSurv(**hyperparams)
        metrics = network.train(train_data, n_epochs = num_epochs, logger=logger, 
            update_fn = update_fn, verbose = False)

        result = network.get_concordance_index(**valid_data)
        main_logger.info('Run id: %s | %s | C-Index: %f | Train Loss %f' % (run_id, str(hyperparams), result, metrics['loss'][-1][1]))
        return result

    return train_deepsurv

In [88]:
NUM_EPOCHS = 1000
NUM_FOLDS = 10
logdir='/home/arash/VRdataCleaning/logs'
x = data['x']
e = data['e']
t = data['t']
y = np.column_stack((e, t))


In [89]:
#    global main_logger
main_logger = load_logger(logdir)


In [90]:
#    main_logger.debug('Loading dataset: ' + args.dataset)
box_constraints = load_box_constraints('/home/arash/VRdataCleaning/box_constraints.0.json')


In [91]:
opt_fxn = get_objective_function(NUM_EPOCHS, logdir, 
                                 utils.get_optimizer_from_str('adam'))

In [92]:
opt_fxn = optunity.cross_validated(x=x, y=y, num_folds=NUM_FOLDS)(opt_fxn)

In [ ]:
#    main_logger.debug('Maximizing C-Index. Num_iterations: %d' % args.num_evals)
opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=100,
        solver_name='sobol',
        **box_constraints)

#    main_logger.debug('Optimal Parameters: ' + str(opt_params))
#    main_logger.debug('Saving Call log...')

2019-03-27 14:39:52,856 - __main__ - INFO - Run id: -5505175062005344475_d30405af-c805-4bfb-b3d7-eb81fe9959cf | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.475282 | Train Loss 12.300897
2019-03-27 14:39:52,856 - __main__ - INFO - Run id: -5505175062005344475_d30405af-c805-4bfb-b3d7-eb81fe9959cf | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.475282 | Train Loss 12.300897
2019-03-27 14:39:52,856 - __main__ - INFO - Run id: -5505175062005344475_d30405af-c805-4bfb-b3d7-eb81fe9959cf | {'batch_norm': False, 'activation': 'r

2019-03-27 14:45:05,189 - __main__ - INFO - Run id: -5505175062005344475_d6906d21-5bc7-437d-a074-4a660c499042 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.500959 | Train Loss 12.242240
2019-03-27 14:45:05,189 - __main__ - INFO - Run id: -5505175062005344475_d6906d21-5bc7-437d-a074-4a660c499042 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.500959 | Train Loss 12.242240
2019-03-27 14:45:05,189 - __main__ - INFO - Run id: -5505175062005344475_d6906d21-5bc7-437d-a074-4a660c499042 | {'batch_norm': False, 'activation': 'r

2019-03-27 14:52:59,396 - __main__ - INFO - Run id: -5505175062005344475_5ea0db1d-fc66-45e7-9d50-345f663d6d80 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.557678 | Train Loss 12.512341
2019-03-27 14:52:59,396 - __main__ - INFO - Run id: -5505175062005344475_5ea0db1d-fc66-45e7-9d50-345f663d6d80 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.557678 | Train Loss 12.512341
2019-03-27 14:52:59,396 - __main__ - INFO - Run id: -5505175062005344475_5ea0db1d-fc66-45e7-9d50-345f663d6d80 | {'batch_norm': False, 'activation': 'r

2019-03-27 14:58:13,392 - __main__ - INFO - Run id: -5505175062005344475_6236127c-a2e4-4ade-b1ff-698d0d2e34ea | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.475806 | Train Loss 13.957478
2019-03-27 14:58:13,392 - __main__ - INFO - Run id: -5505175062005344475_6236127c-a2e4-4ade-b1ff-698d0d2e34ea | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00034337890625, 'learning_rate': 4.5914845258188133e-07, 'n_in': 13, 'momentum': 0.9443193359375, 'L2_reg': 3.1662763671875, 'dropout': 0.14447265625, 'hidden_layers_sizes': [57, 57, 57]} | C-Index: 0.475806 | Train Loss 13.957478
2019-03-27 14:58:13,392 - __main__ - INFO - Run id: -5505175062005344475_6236127c-a2e4-4ade-b1ff-698d0d2e34ea | {'batch_norm': False, 'activation': 'r

2019-03-27 15:06:43,638 - __main__ - INFO - Run id: -8017437063962878789_6c38b71c-f182-4ff6-8ea3-dad04d5c3761 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00083837890625, 'learning_rate': 4.384833680989119e-05, 'n_in': 6, 'momentum': 0.8700693359375, 'L2_reg': 0.7160263671874999, 'dropout': 0.04547265625, 'hidden_layers_sizes': [82, 82, 82, 82]} | C-Index: 0.509784 | Train Loss 7.474672
2019-03-27 15:06:43,638 - __main__ - INFO - Run id: -8017437063962878789_6c38b71c-f182-4ff6-8ea3-dad04d5c3761 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00083837890625, 'learning_rate': 4.384833680989119e-05, 'n_in': 6, 'momentum': 0.8700693359375, 'L2_reg': 0.7160263671874999, 'dropout': 0.04547265625, 'hidden_layers_sizes': [82, 82, 82, 82]} | C-Index: 0.509784 | Train Loss 7.474672
2019-03-27 15:06:43,638 - __main__ - INFO - Run id: -8017437063962878789_6c38b71c-f182-4ff6-8ea3-dad04d5c3761 | {'batch_norm': False, 'activat

...

2019-03-27 15:22:55,620 - __main__ - INFO - Run id: -8017437063962878789_6d1aa62d-259b-4ba8-bf21-90569282c755 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00083837890625, 'learning_rate': 4.384833680989119e-05, 'n_in': 6, 'momentum': 0.8700693359375, 'L2_reg': 0.7160263671874999, 'dropout': 0.04547265625, 'hidden_layers_sizes': [82, 82, 82, 82]} | C-Index: 0.583059 | Train Loss 7.428693
2019-03-27 15:22:55,620 - __main__ - INFO - Run id: -8017437063962878789_6d1aa62d-259b-4ba8-bf21-90569282c755 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00083837890625, 'learning_rate': 4.384833680989119e-05, 'n_in': 6, 'momentum': 0.8700693359375, 'L2_reg': 0.7160263671874999, 'dropout': 0.04547265625, 'hidden_layers_sizes': [82, 82, 82, 82]} | C-Index: 0.583059 | Train Loss 7.428693
2019-03-27 15:22:55,620 - __main__ - INFO - Run id: -8017437063962878789_6d1aa62d-259b-4ba8-bf21-90569282c755 | {'batch_norm': False, 'activat

2019-03-27 15:29:23,180 - __main__ - INFO - Run id: -8017437063962878789_4774ca2a-f3ee-4987-8c13-cf1f24776dc9 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00083837890625, 'learning_rate': 4.384833680989119e-05, 'n_in': 6, 'momentum': 0.8700693359375, 'L2_reg': 0.7160263671874999, 'dropout': 0.04547265625, 'hidden_layers_sizes': [82, 82, 82, 82]} | C-Index: 0.520316 | Train Loss 7.609415
2019-03-27 15:29:23,180 - __main__ - INFO - Run id: -8017437063962878789_4774ca2a-f3ee-4987-8c13-cf1f24776dc9 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00083837890625, 'learning_rate': 4.384833680989119e-05, 'n_in': 6, 'momentum': 0.8700693359375, 'L2_reg': 0.7160263671874999, 'dropout': 0.04547265625, 'hidden_layers_sizes': [82, 82, 82, 82]} | C-Index: 0.520316 | Train Loss 7.609415
2019-03-27 15:29:23,180 - __main__ - INFO - Run id: -8017437063962878789_4774ca2a-f3ee-4987-8c13-cf1f24776dc9 | {'batch_norm': False, 'activat

2019-03-27 15:39:14,260 - __main__ - INFO - Run id: -1231369156044878803_a3ad05ef-89da-4628-8139-faf6395bb258 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00059087890625, 'learning_rate': 4.486969578072787e-06, 'n_in': 9, 'momentum': 0.9071943359375, 'L2_reg': 1.9411513671874998, 'dropout': 0.19397265625000001, 'hidden_layers_sizes': [94, 94, 94, 94]} | C-Index: 0.536492 | Train Loss 10.473975
2019-03-27 15:39:14,260 - __main__ - INFO - Run id: -1231369156044878803_a3ad05ef-89da-4628-8139-faf6395bb258 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00059087890625, 'learning_rate': 4.486969578072787e-06, 'n_in': 9, 'momentum': 0.9071943359375, 'L2_reg': 1.9411513671874998, 'dropout': 0.19397265625000001, 'hidden_layers_sizes': [94, 94, 94, 94]} | C-Index: 0.536492 | Train Loss 10.473975
2019-03-27 15:39:14,260 - __main__ - INFO - Run id: -1231369156044878803_a3ad05ef-89da-4628-8139-faf6395bb258 | {'batch_norm': F

2019-03-27 15:45:59,788 - __main__ - INFO - Run id: -1231369156044878803_2c23e34c-251d-4d6d-bf36-ddcf9d806e47 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00059087890625, 'learning_rate': 4.486969578072787e-06, 'n_in': 9, 'momentum': 0.9071943359375, 'L2_reg': 1.9411513671874998, 'dropout': 0.19397265625000001, 'hidden_layers_sizes': [94, 94, 94, 94]} | C-Index: 0.546825 | Train Loss 10.655000
2019-03-27 15:45:59,788 - __main__ - INFO - Run id: -1231369156044878803_2c23e34c-251d-4d6d-bf36-ddcf9d806e47 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00059087890625, 'learning_rate': 4.486969578072787e-06, 'n_in': 9, 'momentum': 0.9071943359375, 'L2_reg': 1.9411513671874998, 'dropout': 0.19397265625000001, 'hidden_layers_sizes': [94, 94, 94, 94]} | C-Index: 0.546825 | Train Loss 10.655000
2019-03-27 15:45:59,788 - __main__ - INFO - Run id: -1231369156044878803_2c23e34c-251d-4d6d-bf36-ddcf9d806e47 | {'batch_norm': F

2019-03-27 15:52:49,899 - __main__ - INFO - Run id: -1231369156044878803_ad9a6f63-aadf-4e07-bf38-6c8342c5daf7 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00059087890625, 'learning_rate': 4.486969578072787e-06, 'n_in': 9, 'momentum': 0.9071943359375, 'L2_reg': 1.9411513671874998, 'dropout': 0.19397265625000001, 'hidden_layers_sizes': [94, 94, 94, 94]} | C-Index: 0.530139 | Train Loss 10.604929
2019-03-27 15:52:49,899 - __main__ - INFO - Run id: -1231369156044878803_ad9a6f63-aadf-4e07-bf38-6c8342c5daf7 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.00059087890625, 'learning_rate': 4.486969578072787e-06, 'n_in': 9, 'momentum': 0.9071943359375, 'L2_reg': 1.9411513671874998, 'dropout': 0.19397265625000001, 'hidden_layers_sizes': [94, 94, 94, 94]} | C-Index: 0.530139 | Train Loss 10.604929
2019-03-27 15:56:18,606 - __main__ - INFO - Run id: -1231369156044878803_15beb23b-43a8-4e41-9737-2a6d8412eab8 | {'batch_norm': F

In [85]:
print(call_log._asdict())

OrderedDict([('optimum', 0.6102046723178433), ('stats', OrderedDict([('num_evals', 30), ('time', 57160.077339448966)])), ('call_log', {'args': {'learning_rate': [-5.5143359375, -3.5343359375, -5.6380859375000005, -3.6580859375, -6.628085937500001, -4.6480859375, -6.133085937500001, -4.1530859375, -5.1430859375, -3.1630859375, -6.8755859375, -4.8955859375, -5.8855859375, -3.9055859375, -5.3905859375, -3.4105859375, -6.3805859375, -4.400585937500001, -6.7827734375, -4.802773437500001, -5.7927734375, -3.8127734375, -5.2977734375, -3.3177734375, -6.2877734375, -4.3077734375, -5.545273437500001, -3.5652734375, -6.535273437500001, -4.5552734375], 'lr_decay': [0.000616982421875, 0.00012198242187500001, 9.104492187500001e-05, 0.000586044921875, 0.000833544921875, 0.00033854492187500003, 0.00046229492187500003, 0.000957294921875, 0.000709794921875, 0.000214794921875, 0.000152919921875, 0.000647919921875, 0.000895419921875, 0.000400419921875, 0.000276669921875, 0.0007716699218750001, 0.000524169

In [ ]:
save_call_log(os.path.join(logdir, 'optunity_log_%s.pkl' % (str(uuid.uuid4()))), call_log._asdict())

In [86]:
opt_params

{'lr_decay': 0.00014518554687500002,
 'learning_rate': -4.3077734375,
 'num_nodes': 81.23583984375,
 'num_layers': 5.071376953125,
 'n_in': 17.242138671875,
 'momentum': 0.8174272460937501,
 'L2_reg': 1.0175219726562499,
 'dropout': 0.11875585937500001}

In [ ]:
opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=10,
        solver_name='sobol',
        **box_constraints)

In [ ]:
x_train[:,ReliefAvg.sort_values(['Importance'],ascending=False).index[0:n_in]]

In [ ]:
ReliefAvg = pd.read_pickle('/home/arash/VRdataCleaning/DeepSurv/deepsurv/ReliefAvg.pkl')
        

In [ ]:
x_train=x_train[:,ReliefAvg.sort_values(['Importance'],ascending=False).index[0:n_in]]